In [23]:
#Importing OpenCV Library for basic image processing functions
import cv2
# Numpy for array related functions
import numpy as np
# Dlib for deep learning based Modules and face landmark detection
import dlib
#face_utils for basic operations of conversion
from imutils import face_utils
from scipy.spatial import distance

In [38]:
#Initializing the camera and taking the instance
cap = cv2.VideoCapture(0)

#Initializing the face detector and landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#status marking for current state
happy = 0
sad = 0
neutral = 0
status = ""
color = (0,0,0)

In [39]:
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[3], mouth[9])  # 51-59
    B = distance.euclidean(mouth[2], mouth[10]) # 50-60
    C = distance.euclidean(mouth[4], mouth[8])  # 52-58
    avg = (A + B + C) / 3
    D = distance.euclidean(mouth[0], mouth[6])  # 48-54

    MAR = avg / D
    return MAR

    # Inside your main loop after detecting landmarks:
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    #detected face in faces array
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        face_frame = frame.copy()
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        landmarks = predictor(gray, face) # Predict facial landmarks
        landmarks = face_utils.shape_to_np(landmarks)

        MAR = mouth_aspect_ratio(landmarks[48:68])

        SMILE_THRESHOLD = 0.3
        NEUTRAL_THRESHOLD = 0.2

        if MAR > SMILE_THRESHOLD:
            status = "Smiling!"
            color = (0, 255, 0)
        elif MAR < NEUTRAL_THRESHOLD:
            status = "Sad!"
            color = (255, 0, 0)
        else:
            status = "Neutral"
            color = (0, 0, 255)
    
        cv2.putText(frame, status, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color,3)

        for n in range(0, 68):
            (x,y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

    cv2.imshow("Frame", frame)
    cv2.imshow("Result of detector", face_frame)
    #key = cv2.waitKey(1)
    #if key == 27 & 0xFF == ord('q'):
      #  break
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()